In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dtree').getOrCreate()

In [3]:
df = spark.read.csv('FileStore/tables/College.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
df.columns

Out[ 6 ]: 
['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [7]:
assembler = VectorAssembler(inputCols= ['Apps','Accept', 'Enroll', 
                                        'Top10perc', 'Top25perc', 'F_Undergrad', 
                                        'P_Undergrad', 'Outstate', 'Room_Board', 
                                        'Books', 'Personal', 'PhD', 
                                        'Terminal', 'S_F_Ratio', 'perc_alumni',
                                        'Expend', 'Grad_Rate'],
                            outputCol='features')

In [8]:
output= assembler.transform(df)

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [11]:
output_fixed = indexer.fit(output).transform(output)

In [12]:
output_fixed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [13]:
final_data = output_fixed.select(['features', 'PrivateIndex'])

In [14]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [15]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [16]:
from pyspark.ml import Pipeline

In [17]:
dt_clf = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rf_clf = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt_clf = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [18]:
dt_model = dt_clf.fit(train_data)
rf_model = rf_clf.fit(train_data)
gbt_model = gbt_clf.fit(train_data)

In [19]:
dt_preds = dt_model.transform(test_data)
rf_preds = rf_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
my_bin_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex')

In [22]:
print('DTC')
my_bin_eval.evaluate(dt_preds)

DTC
 Out[ 22 ]: 0.9305317324185247

In [23]:
print('RFC')
my_bin_eval.evaluate(rf_preds)

RFC
 Out[ 23 ]: 0.9708404802744424

In [24]:
my_pred_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex', rawPredictionCol='prediction')

In [25]:
gbt_preds.show()

+--------------------+------------+--------------------+--------------------+----------+
 features|PrivateIndex| rawPrediction| probability|prediction|
+--------------------+------------+--------------------+--------------------+----------+
[81.0,72.0,51.0,3...| 0.0|[1.07865722302084...|[0.89635030874916...| 0.0|
[100.0,90.0,35.0,...| 0.0|[1.54695293787433...|[0.95664066731342...| 0.0|
[174.0,146.0,88.0...| 0.0|[1.52781694378073...|[0.95502513526715...| 0.0|
[193.0,146.0,55.0...| 0.0|[1.38436510072148...|[0.94096248606176...| 0.0|
[212.0,197.0,91.0...| 0.0|[1.54694470750408...|[0.95663998452827...| 0.0|
[233.0,233.0,153....| 1.0|[-2.9599104128975...|[0.00267848777403...| 1.0|
[321.0,318.0,172....| 0.0|[-0.1673305381192...|[0.41710694559062...| 1.0|
[325.0,260.0,86.0...| 0.0|[1.54200175296557...|[0.95622806157631...| 0.0|
[348.0,281.0,127....| 0.0|[1.54695293787433...|[0.95664066731342...| 0.0|
[355.0,300.0,142....| 0.0|[1.54694470750408...|[0.95663998452827...| 0.0|
[361.0,321.0,185....| 0.0|[1.53439777818779...|[0.95558708337045...| 0.0|
[374.0,280.0,185....| 0.0|[1.44157071112319...|[0.94700673736666...| 0.0|
[380.0,237.0,104....| 0.0|[1.54694470750408...|[0.95663998452827...| 0.0|
[385.0,340.0,193....| 0.0|[1.50997403745163...|[0.95346722175388...| 0.0|
[392.0,351.0,155....| 0.0|[1.54694470750408...|[0.95663998452827...| 0.0|
[417.0,349.0,137....| 0.0|[1.51103903011970...|[0.95356163260800...| 0.0|
[420.0,293.0,93.0...| 0.0|[1.43642081347292...|[0.94648745615854...| 0.0|
[441.0,369.0,172....| 1.0|[1.38008861575843...|[0.94048555488478...| 0.0|
[443.0,330.0,151....| 0.0|[1.55722811500496...|[0.95748512740248...| 0.0|
[444.0,359.0,122....| 0.0|[1.54694470750408...|[0.95663998452827...| 0.0|
+--------------------+------------+--------------------+--------------------+----------+
only showing top 20 rows

In [26]:
print('GBTC')
my_pred_eval.evaluate(gbt_preds)

GBTC
 Out[ 26 ]: 0.8801293046576065

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [29]:
acc_eval.evaluate(rf_preds)

Out[ 29 ]: 0.9183673469387755

In [30]:
acc_eval.evaluate(dt_preds)

Out[ 30 ]: 0.9030612244897959

In [31]:
acc_eval.evaluate(gbt_preds)

Out[ 31 ]: 0.9030612244897959